In [1]:
import os
import logging
import pickle
import argparse
from tqdm import tqdm  # printing progress
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

import pandas

In [2]:
path = "dbpedia_csv/"

In [3]:
_, _, x_train = pickle.load(open(path + "/train_preprocessed.pickle", "rb"))

_, _, x_test = pickle.load(open(path + "/test_preprocessed.pickle", "rb"))

In [5]:
doc_list = x_train + x_test
num_pass = 0

In [10]:
# embedding_size = ARGS.w2v_embed_size  # Dimension of the embedding vector.
# window = ARGS.w2v_window  # How many words to consider left and right.

# "-we", "--w2v_embed_size",

#model = Word2Vec(doc_list, size=50, workers=3, window =3, sg = 1)
w2v_model = Word2Vec(doc_list, vector_size=50, window=3, min_count=1, workers=4)

In [13]:


split_examples =[x.split() for x in doc_list]
split_examples[1]

['schwanstabilo',
 'german',
 'maker',
 'pens',
 'writing',
 'colouring',
 'cosmetics',
 'well',
 'markers',
 'highlighters',
 'office',
 'use',
 'worlds',
 'largest',
 'manufacturer',
 'highlighter',
 'pens',
 'stabilo',
 'boss']

In [17]:
w2v_model.train(split_examples, total_examples= len(doc_list), epochs=20)

(1775483, 337947460)

In [21]:
import numpy as np

def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += w2v_model.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

wordvec_arrays = np.zeros((len(split_examples), 50)) 
for i in range(len(split_examples)):
    wordvec_arrays[i,:] = word_vector(split_examples[i], 50)


NameError: name 'pd' is not defined

In [22]:
wordvec_df = pandas.DataFrame(wordvec_arrays)
wordvec_df.shape

(630000, 50)

In [26]:
y_train, _, _ = pickle.load(open(path + "/train_preprocessed.pickle", "rb"))

y_test, _, _ = pickle.load(open(path + "/test_preprocessed.pickle", "rb"))

In [27]:
len(y_train + y_test)

630000

In [28]:
len(y_train)

560000

In [29]:
wordvec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,-0.409411,-0.481766,-0.449141,1.091868,-0.768192,-0.060451,-0.311895,-0.140397,0.344459,0.660494,...,0.992893,-0.221473,-0.568633,0.686275,-0.275348,0.551287,0.613449,-0.187132,-0.870731,0.044502
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
x_train = wordvec_df[:560000]

In [32]:
x_test = wordvec_df[560000:]

In [24]:
from sklearn import svm

import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [34]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 11).fit(x_train, y_train)

accuracy = knn.score(x_test, y_test)
print(accuracy)

knn_predictions = knn.predict(x_test)
cm = confusion_matrix(y_test, knn_predictions)

cm

0.08565714285714286


NameError: name 'confusion_matrix' is not defined

In [35]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, knn_predictions)

cm

array([[  17,   14,   17,    4,  104,   19,   36,    2, 4618,    0,   16,
         119,    7,   27],
       [  22,   33,   12,    4,   91,   32,   33,    1, 4583,    0,   22,
         123,    8,   36],
       [  24,   20,   12,    2,  119,   28,   38,    4, 4589,    0,   32,
          80,   10,   42],
       [  13,   13,    8,   32,   62,   26,   17,    3, 4609,    0,   37,
         123,    2,   55],
       [  24,   30,   33,   13,  348,   79,  109,    1, 4140,    0,   43,
         125,    9,   46],
       [  32,   28,    9,   16,  226,  236,   90,    2, 4056,    0,   52,
         158,   21,   74],
       [  44,    9,   24,    5,  248,   65,  127,    1, 4167,    0,   39,
         145,   10,  116],
       [   6,    4,    7,    2,   41,   19,   27,    7, 4780,    0,   10,
          65,    3,   29],
       [   1,   11,    1,    6,   12,   21,    3,    4, 4675,    0,    9,
         208,    0,   49],
       [  12,    4,    9,    2,   55,   11,   30,    2, 4829,    0,   10,
          21,    

In [37]:
w2v_model.save("word2vec_first")

w2v_model.wv.save("word2vec_first_embeddings")